In [1]:
import pandas as pd 
import numpy as np  

In [2]:
colab = False # change to true if running on colab 

# Overview. 

The purpose of this notebook is to predict land value for the purpose of helping consumers fairly acquire loans if they are able. 

The data used in this notebook can be found [here](https://data.vbgov.com/dataset/property-sales/resource/4b8d0176-bb17-43e4-960c-59119ac5b091). The description of the columns are also there. 

In [3]:
# Get data 
import os
if colab: 
    from google.colab import drive
    drive.mount('/content/drive')

In [4]:
if colab: 
    os.chdir('/content/drive/MyDrive/Colab Notebooks')
X = pd.read_csv('./data/Property_Sales.csv')

/Users/lukeanglin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
X.head()

,GPIN,Street Address,City,State,Zip Code,Neighborhood,Land Value,Improvement Value,Total Value,Land USE (Y/N),Sale Date,Sale Price,Document Number,Deed Book,Deed Page
0,24171562220000,NaN,Virginia Beach,VA,NaN,Zero Value Com Elements,0,0,0,No,10/13/2000,150122,000000000000000,4299,28.0
1,15809055970003,3160 Page Ave 400,Virginia Beach,VA,23451-7316,Bayfront/Bayside Condo Dunes,341000,406600,747600,No,07/19/2011,690000,20110719000718380,NaN,NaN
2,24161082000000,1453 Gate Tree Ct 101,Virginia Beach,VA,23454,Apartments,1150000,4814000,5964000,No,07/21/1995,139500,000000000000000,3518,799.0
3,24186722550000,1137 Cedar Point Dr,Virginia Beach,VA,23451-3843,Bay Colony,426800,330300,757100,No,01/04/1999,390000,000000000000000,3980,688.0
4,24049362920000,2224 Kindling Hollow Rd,Virginia Beach,VA,23456-3845,Three Oaks,175000,331200,506200,No,09/17/2021,590000,202103074532,NaN,NaN


# Data Cleaning

I want to get rid of the sale price column as we are looking at total value and those will correlate too much. I want to be able to predict the value without the price. Document number, Deed Book and Page, GPIN, and Street address are unnecessary. City and State as well, because they are the same for every value.

In [6]:
# Drop useless columns
features_to_remove = ['GPIN', 'Street Address', 'Document Number', 'Sale Price', 'City', 'State', 'Deed Book', 'Deed Page']
for feature in features_to_remove: 
    if feature in X.columns:
        X = X.drop(feature, axis=1) 
print(f'Columns left: {X.columns}')

Columns left: Index(['Zip Code', 'Neighborhood', 'Land Value', 'Improvement Value',
       'Total Value', 'Land USE (Y/N)', 'Sale Date'],
      dtype='object')


Split into X and y

In [8]:
# Drop Total Value, set that to y 
if 'Total Value' in X.columns: 
    y = X['Total Value'] 
    X = X.drop('Total Value', axis = 1)

# Convert this to datetime
if 'Sale Date' in X.columns:
    X['Sale Date'] = pd.to_datetime(X['Sale Date'], infer_datetime_format=True)
    # Create year and month columns from datetime 
    X['Year'] = X['Sale Date'].apply(lambda time: time.year)
    X['Month'] = X['Sale Date'].apply(lambda time: time.month)
    X.drop('Sale Date', axis = 1, inplace = True)
X.head()

,Zip Code,Neighborhood,Land Value,Improvement Value,Land USE (Y/N),Year,Month
0,NaN,Zero Value Com Elements,0,0,No,2000.0,10.0
1,23451-7316,Bayfront/Bayside Condo Dunes,341000,406600,No,2011.0,7.0
2,23454,Apartments,1150000,4814000,No,1995.0,7.0
3,23451-3843,Bay Colony,426800,330300,No,1999.0,1.0
4,23456-3845,Three Oaks,175000,331200,No,2021.0,9.0


Let's investigate the data types of each of these. 

* Categorical columns - Zip Code, Land Use
* Numerical columns - Improvement Value, Land Value, Year, Month

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


cat_cols = ['Zip Code', 'Land USE (Y/N)']
num_cols = ['Improvement Value', 'Land Value', 'Year', 'Month']
for col in num_cols: 
    X[col] = pd.to_numeric(X[col], errors = 'coerce')

num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
        ('imputer', SimpleImputer(strategy="median"))
    ])

cat_pipeline = Pipeline([
    ('imputer', DataFrameImputer()), 
    ('encoder', OrdinalEncoder()) 
])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])
# One Hot encoding creates way too many columns, so we'll use a target encoder 
X_cleaned = full_pipeline.fit_transform(X)

In [23]:
X_cleaned.shape

(556191, 6)

In [27]:
X = pd.DataFrame(X_cleaned, columns = num_cols + cat_cols)


## Feature Correlations 

Hover over the boxes to view the actual value. We. learn from this matrix that improvement value and total value have almost a perfect positive correlation.

In [28]:
# Let's visualize the correlation between features now. 
from pandas.plotting import scatter_matrix
import plotly.express as px 
from copy import copy 

X_with_labels = copy(X)
X_with_labels['Total Value'] = y
# scatter_matrix(data[data.columns], figsize=(20, 15))
px.imshow(X_with_labels.corr())

# Simple Linear Regression

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
reg = LinearRegression() 
reg.fit(X, y) 
cross_val_score(reg, X, y) 

array([1., 1., 1., 1., 1.])